In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display as ipy_display


conn = sqlite3.connect("Chinook.sqlite")


query = """
SELECT 
    Customer.LastName, 
    Customer.FirstName, 
    Track.Name AS Name, 
    Album.Title AS Title
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName ASC, Customer.FirstName ASC;
"""


df = pd.read_sql_query(query, conn)


conn.close()


def paginate_data(df, page_num, rows_per_page):
   
    start_idx = (page_num - 1) * rows_per_page
    end_idx = start_idx + rows_per_page
    
    ipy_display(df.iloc[start_idx:end_idx])


rows_per_page = 10
max_pages = len(df) // rows_per_page + (1 if len(df) % rows_per_page > 0 else 0)
page_num = 1


def on_prev_button_clicked(change):
    global page_num
    if page_num > 1:
        page_num -= 1
    update_display()

def on_next_button_clicked(change):
    global page_num
    if page_num < max_pages:
        page_num += 1
    update_display()


def update_display():
    print(f"\n\U0001F4CC Page {page_num} of the dataset:")
    paginate_data(df, page_num, rows_per_page)


update_display()


prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")


prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)


widget_box = widgets.HBox([prev_button, next_button])
display(widget_box)


print("\n\U0001F4CC Dataset Shape (Rows, Columns):", df.shape)


unique_customers = df[['LastName', 'FirstName']].drop_duplicates()
print("\n\U0001F4CC Unique Customers Count:", unique_customers.shape[0])


customer_purchases = df.groupby(['LastName', 'FirstName'], as_index=False).agg(Total_Purchases=('Name', 'count'))


print("\n\U0001F4CC Top 10 Customers by Number of Purchases:")
display(customer_purchases.sort_values(by='Total_Purchases', ascending=False).head(10))


df.to_csv("customer_purchases.csv", index=False)
print("\n✅ Data exported to 'customer_purchases.csv'")
